<a href="https://colab.research.google.com/github/Jhancyamarsingh17/Automated-medical-report-generation-and-classification-/blob/main/Inception_model_for_medical_image_captioning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Importing Dependencies

In [25]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [26]:
import tensorflow as tf
import matplotlib.pyplot as plt
import collections
import random
import numpy as np
import os
import time
import json
from PIL import Image

### Loading in the model

In [27]:
## Visualizing the dataset 
import pandas as pd
df = pd.read_csv("/content/gdrive/MyDrive/Data606_Capstone_project_folder/data/indiana_reports.csv")
df.head()

,uid,MeSH,Problems,image,indication,comparison,findings,impression
0,1,normal,normal,Xray Chest PA and Lateral,Positive TB test,None.,The cardiac silhouette and mediastinum size ar...,Normal chest x-XXXX.
1,2,Cardiomegaly/borderline;Pulmonary Artery/enlarged,Cardiomegaly;Pulmonary Artery,"Chest, 2 views, frontal and lateral",Preop bariatric surgery.,None.,Borderline cardiomegaly. Midline sternotomy XX...,No acute pulmonary findings.
2,3,normal,normal,Xray Chest PA and Lateral,"rib pain after a XXXX, XXXX XXXX steps this XX...",NaN,NaN,"No displaced rib fractures, pneumothorax, or p..."
3,4,"Pulmonary Disease, Chronic Obstructive;Bullous...","Pulmonary Disease, Chronic Obstructive;Bullous...","PA and lateral views of the chest XXXX, XXXX a...",XXXX-year-old XXXX with XXXX.,None available,There are diffuse bilateral interstitial and a...,1. Bullous emphysema and interstitial fibrosis...
4,5,Osteophyte/thoracic vertebrae/multiple/small;T...,Osteophyte;Thickening;Lung,Xray Chest PA and Lateral,Chest and nasal congestion.,NaN,The cardiomediastinal silhouette and pulmonary...,No acute cardiopulmonary abnormality.


In [28]:
dff = pd.read_csv("/content/gdrive/MyDrive/Data606_Capstone_project_folder/data/indiana_projections.csv")
dff.head()

,uid,filename,projection
0,1,1_IM-0001-4001.dcm.png,Frontal
1,1,1_IM-0001-3001.dcm.png,Lateral
2,2,2_IM-0652-1001.dcm.png,Frontal
3,2,2_IM-0652-2001.dcm.png,Lateral
4,3,3_IM-1384-1001.dcm.png,Frontal


In [29]:
dataframe = pd.merge(df, dff, on =['uid'])

In [31]:
len(dataframe)

7466

In [32]:
dataframe.isnull().sum()

uid              0
MeSH             0
Problems         0
image            0
indication     159
comparison    1157
findings       997
impression      52
filename         0
projection       0
dtype: int64

In [33]:
import string
import regex as re 
import nltk
from nltk.tokenize import word_tokenize

# nltk.download('stopwords')
# from nltk.corpus import stopwords
nltk.download('punkt')

def decontraction(doc):
    docs = re.sub(r"won't", "will not", doc)
    docs = re.sub(r"can\'t", "can not", docs)
    docs = re.sub(r"n\'t", " not", docs)
    docs = re.sub(r"\'re", " are", docs)
    docs = re.sub(r"\'s", " is", docs)
    docs = re.sub(r"\'d", " would", docs)
    docs = re.sub(r"\'ll", " will", docs)
    docs = re.sub(r"\'t", " not", docs)
    docs = re.sub(r"\'ve", " have", docs)
    docs = re.sub(r"\'m", " am",docs)
    docs = re.sub(r"n\'t", " not", docs)
    docs = re.sub(r"\'re", " are", docs)
    docs = re.sub(r"\'s", " is", docs)
    docs = re.sub(r"\'d", " would", docs)
    docs = re.sub(r"\'ll", " will", docs)
    docs = re.sub(r"\'t", " not", docs)
    docs = re.sub(r"\'ve", " have",docs)
    docs = re.sub(r"\'m", " am", docs)
    docs = re.sub(r"\*+", "abuse", docs)
    return docs

def remove_stopwords(data):
  all_stopwords = stopwords.words('english')
  all_stopwords.remove('not')# removing the "not" from the stopwords
  all_stopwords.remove('no')
  text_tokens = word_tokenize(data) # tokenizing the sentence into word
  tokens_without_sw = [word for word in text_tokens if not word in all_stopwords]
  return " ".join(tokens_without_sw)

def text_preprocess(data):
  clean_text=[]
  text=str(data)
  # for i in (data.values):
  #   text = BeautifulSoup(i, 'lxml').get_text()
  text=text.lower()# making the text in lower case
  text=re.sub(r"[|\'||\\||/|?|$|.|.|,||;|><!&)(:`\n\t@#=+%^*~-]","",text)# removing the special character in text
  text=re.sub(r"xxxx","",text)# Removing the irrelavent XXXX words
  text=re.sub(r'xxx',"",text)
  text=re.sub(r'xx',"",text)
  text=re.sub(r'[0-9]',"",text)# Removing the number in text
  text = re.sub(r"yearold", "", text)
  text= re.sub('\s+',' ',text)
  text=re.sub("'",'',text)# Removing apoustahe
  text=re.sub(r'\.\.+', '.',text)# Removing multiple full stop
  text=re.sub(r'  ',"",text)
  text=decontraction(text)
  # text=remove_stopwords(text)
  
  return text
#  
dataframe["indication"]=dataframe["indication"].apply(lambda x:text_preprocess(x))
dataframe["findings"]=dataframe["findings"].apply(lambda x:text_preprocess(x))
dataframe["impression"]=dataframe["impression"].apply(lambda x:text_preprocess(x))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [34]:
dataframe.head()

,uid,MeSH,Problems,image,indication,comparison,findings,impression,filename,projection
0,1,normal,normal,Xray Chest PA and Lateral,positive tb test,None.,the cardiac silhouette and mediastinum size ar...,normal chest x,1_IM-0001-4001.dcm.png,Frontal
1,1,normal,normal,Xray Chest PA and Lateral,positive tb test,None.,the cardiac silhouette and mediastinum size ar...,normal chest x,1_IM-0001-3001.dcm.png,Lateral
2,2,Cardiomegaly/borderline;Pulmonary Artery/enlarged,Cardiomegaly;Pulmonary Artery,"Chest, 2 views, frontal and lateral",preop bariatric surgery,None.,borderline cardiomegaly midline sternotomy enl...,no acute pulmonary findings,2_IM-0652-1001.dcm.png,Frontal
3,2,Cardiomegaly/borderline;Pulmonary Artery/enlarged,Cardiomegaly;Pulmonary Artery,"Chest, 2 views, frontal and lateral",preop bariatric surgery,None.,borderline cardiomegaly midline sternotomy enl...,no acute pulmonary findings,2_IM-0652-2001.dcm.png,Lateral
4,3,normal,normal,Xray Chest PA and Lateral,rib pain after a steps this pain to r back r e...,NaN,nan,no displaced rib fractures pneumothorax or ple...,3_IM-1384-1001.dcm.png,Frontal


In [35]:
df = dataframe[['filename','indication','findings','impression','projection']]

In [37]:
image_grp = []
for i in range(df.shape[0]):
  #print(req_df['filename'][i].split('.')[0])
  image_grp.append('_'.join(df['filename'][i].split('-')[0:-1]))
df['image_grp'] = image_grp

<ipython-input-37-231327161734>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['image_grp'] = image_grp


In [38]:
df.head()

,filename,indication,findings,impression,projection,image_grp
0,1_IM-0001-4001.dcm.png,positive tb test,the cardiac silhouette and mediastinum size ar...,normal chest x,Frontal,1_IM_0001
1,1_IM-0001-3001.dcm.png,positive tb test,the cardiac silhouette and mediastinum size ar...,normal chest x,Lateral,1_IM_0001
2,2_IM-0652-1001.dcm.png,preop bariatric surgery,borderline cardiomegaly midline sternotomy enl...,no acute pulmonary findings,Frontal,2_IM_0652
3,2_IM-0652-2001.dcm.png,preop bariatric surgery,borderline cardiomegaly midline sternotomy enl...,no acute pulmonary findings,Lateral,2_IM_0652
4,3_IM-1384-1001.dcm.png,rib pain after a steps this pain to r back r e...,nan,no displaced rib fractures pneumothorax or ple...,Frontal,3_IM_1384


In [39]:
df = df.rename(columns={'filename':'image_id'})

In [40]:
df.head()

,image_id,indication,findings,impression,projection,image_grp
0,1_IM-0001-4001.dcm.png,positive tb test,the cardiac silhouette and mediastinum size ar...,normal chest x,Frontal,1_IM_0001
1,1_IM-0001-3001.dcm.png,positive tb test,the cardiac silhouette and mediastinum size ar...,normal chest x,Lateral,1_IM_0001
2,2_IM-0652-1001.dcm.png,preop bariatric surgery,borderline cardiomegaly midline sternotomy enl...,no acute pulmonary findings,Frontal,2_IM_0652
3,2_IM-0652-2001.dcm.png,preop bariatric surgery,borderline cardiomegaly midline sternotomy enl...,no acute pulmonary findings,Lateral,2_IM_0652
4,3_IM-1384-1001.dcm.png,rib pain after a steps this pain to r back r e...,nan,no displaced rib fractures pneumothorax or ple...,Frontal,3_IM_1384


In [41]:
frontal_df = df[df['projection'] == 'Frontal']
frontal_df.head()

,image_id,indication,findings,impression,projection,image_grp
0,1_IM-0001-4001.dcm.png,positive tb test,the cardiac silhouette and mediastinum size ar...,normal chest x,Frontal,1_IM_0001
2,2_IM-0652-1001.dcm.png,preop bariatric surgery,borderline cardiomegaly midline sternotomy enl...,no acute pulmonary findings,Frontal,2_IM_0652
4,3_IM-1384-1001.dcm.png,rib pain after a steps this pain to r back r e...,nan,no displaced rib fractures pneumothorax or ple...,Frontal,3_IM_1384
6,4_IM-2050-1001.dcm.png,with,there are diffuse bilateral interstitial and a...,bullous emphysema and interstitial fibrosis p...,Frontal,4_IM_2050
8,5_IM-2117-1003002.dcm.png,chest and nasal congestion,the cardiomediastinal silhouette and pulmonary...,no acute cardiopulmonary abnormality,Frontal,5_IM_2117


In [42]:
lateral_df = df[df['projection']== 'Lateral']
lateral_df.head()


,image_id,indication,findings,impression,projection,image_grp
1,1_IM-0001-3001.dcm.png,positive tb test,the cardiac silhouette and mediastinum size ar...,normal chest x,Lateral,1_IM_0001
3,2_IM-0652-2001.dcm.png,preop bariatric surgery,borderline cardiomegaly midline sternotomy enl...,no acute pulmonary findings,Lateral,2_IM_0652
5,3_IM-1384-2001.dcm.png,rib pain after a steps this pain to r back r e...,nan,no displaced rib fractures pneumothorax or ple...,Lateral,3_IM_1384
7,4_IM-2050-2001.dcm.png,with,there are diffuse bilateral interstitial and a...,bullous emphysema and interstitial fibrosis p...,Lateral,4_IM_2050
9,5_IM-2117-1004003.dcm.png,chest and nasal congestion,the cardiomediastinal silhouette and pulmonary...,no acute cardiopulmonary abnormality,Lateral,5_IM_2117


In [43]:
image_list = []
for i,ind in zip(frontal_df['image_grp'],frontal_df['image_grp'].index) :
  k = lateral_df[lateral_df['image_grp'] == i]['image_id'].values
  for j in range(len(k)) :
    L = []
    L.append(frontal_df['image_id'][ind])
    L.append(k[j])
    L.append(frontal_df['indication'][ind])
    L.append(frontal_df['findings'][ind])
    L.append(frontal_df['impression'][ind])
    image_list.append(L)
  if len(k) == 0 :
    L = []
    L.append(frontal_df['image_id'][ind])
    L.append(frontal_df['image_id'][ind])
    L.append(frontal_df['indication'][ind])
    L.append(frontal_df['findings'][ind])
    L.append(frontal_df['impression'][ind])
    image_list.append(L)


In [44]:

new_df = pd.DataFrame(image_list,columns=['Frontal','Lateral','indication','findings','impression'])

In [45]:
new_df.shape

(3903, 5)

In [46]:
new_df

,Frontal,Lateral,indication,findings,impression
0,1_IM-0001-4001.dcm.png,1_IM-0001-3001.dcm.png,positive tb test,the cardiac silhouette and mediastinum size ar...,normal chest x
1,2_IM-0652-1001.dcm.png,2_IM-0652-2001.dcm.png,preop bariatric surgery,borderline cardiomegaly midline sternotomy enl...,no acute pulmonary findings
2,3_IM-1384-1001.dcm.png,3_IM-1384-2001.dcm.png,rib pain after a steps this pain to r back r e...,nan,no displaced rib fractures pneumothorax or ple...
3,4_IM-2050-1001.dcm.png,4_IM-2050-2001.dcm.png,with,there are diffuse bilateral interstitial and a...,bullous emphysema and interstitial fibrosis p...
4,5_IM-2117-1003002.dcm.png,5_IM-2117-1004003.dcm.png,chest and nasal congestion,the cardiomediastinal silhouette and pulmonary...,no acute cardiopulmonary abnormality
...,...,...,...,...,...
3898,3995_IM-2046-1001.dcm.png,3995_IM-2046-2001.dcm.png,nausea vomiting x weeks dialysis patient,the cardiomediastinal silhouette and pulmonary...,interval resolution of bibasilar airspace dis...
3899,3996_IM-2047-1001.dcm.png,3996_IM-2047-3001.dcm.png,nan,the lungs are clear heart size is normal no pn...,clear lungs no acute cardiopulmonary abnormality
3900,3997_IM-2048-1001.dcm.png,3997_IM-2048-1002.dcm.png,male with positive ppd,heart size within normal limits small nodular ...,no acute findings no evidence for active tb
3901,3998_IM-2048-1001.dcm.png,3998_IM-2048-1002.dcm.png,tuberculosis positive ppd,nan,heart size is normal and the lungs are clear


In [47]:
new_df['findings'] = new_df['findings'].replace('nan',np.nan)

In [48]:
new_df['indication'] = new_df['indication'].replace('nan',np.nan)
new_df['impression'] = new_df['impression'].replace('nan',np.nan)

In [49]:
new_df.isnull().sum()

Frontal         0
Lateral         0
indication     88
findings      523
impression     31
dtype: int64

In [50]:
new_df.to_csv('/content/gdrive/MyDrive/Capstone/final_df.csv',index=False)